In [2]:
!pip install findspark
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 6.6 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 10.7 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=b04ce80cf439fde31d21a0f6ca76c15f636f33a22e34d6a05bd8153dcf1bae09
  Stored in directory: /home/codespace/.cache/pip/wheels/9c/aa/b1/8433fd8b1afe7eb31196cc74a42cd778bcb52636a428da079d
Successfully built pyspark


In [71]:
import findspark
from pyspark.sql import SparkSession
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import re

In [97]:
spark_session = SparkSession.builder.appName('data_film').getOrCreate()

In [98]:
data = spark_session.read.option("delimiter", "\t").csv('data_film.txt', header=False)
data = data.toDF('titre', 'date', 'duree', 'type', 'note', 'nombre avis', 'avis')

# Preprocess

In [99]:
from pyspark.sql import functions as F

In [100]:
data.printSchema()

root
 |-- titre: string (nullable = true)
 |-- date: string (nullable = true)
 |-- duree: string (nullable = true)
 |-- type: string (nullable = true)
 |-- note: string (nullable = true)
 |-- nombre avis: string (nullable = true)
 |-- avis: string (nullable = true)



In [101]:
# variable avis traitement
data = data.withColumn("avis", F.regexp_replace(F.col("avis"), "[\['\"0-9]" , ""))

In [102]:
# traitment de la variable type
data = data.withColumn("type", F.regexp_replace(F.col("type"), "[\[\]',]", ""))


In [103]:
# ne garder que l'heure pour des besoin de modélisation 
data = data.withColumn('duree', F.substring("duree", 1, 2))

In [104]:
def recuperation_mois() : 

    mois = []
    for date in data.select(F.collect_list("date")).first()[0] : # liste des mois 
        regex_date = re.findall(r'[^0-9-]+', date) 
        mois.append(regex_date[0]) 

    return set(mois)
        
recuperation_mois()

{' août ',
 ' avril ',
 ' décembre ',
 ' février ',
 ' janvier ',
 ' juillet ',
 ' juin ',
 ' mai ',
 ' novembre ',
 ' octobre ',
 ' septembre '}

In [105]:
# afin de passer du format francais a un format usuel 
data = data.withColumn("date", F.regexp_replace('date','août', "08"))
data = data.withColumn("date", F.regexp_replace('date','avril', "04"))
data = data.withColumn("date", F.regexp_replace('date','décembre', "12"))
data = data.withColumn("date", F.regexp_replace('date','février', "02"))
data = data.withColumn("date", F.regexp_replace('date','janvier', "01"))
data = data.withColumn("date", F.regexp_replace('date','juillet', "07"))
data = data.withColumn("date", F.regexp_replace('date','juin', "06"))
data = data.withColumn("date", F.regexp_replace('date','mai', "05"))
data = data.withColumn("date", F.regexp_replace('date','novembre', "11"))
data = data.withColumn("date", F.regexp_replace('date','octobre', "10"))
data = data.withColumn("date", F.regexp_replace('date','septembre', "09"))

In [106]:
# changer le format des dates  
spark_session.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY") # car ne veut pas reconnaitre le format de base 
data = data.withColumn("date", F.to_date("date", "dd-MM-yyyy"))